# Intro

In [30]:
import pandas as pd
import quandl
import math
# numpy is a computing library. impporting to be able to use arrays
import numpy  as np 
# preprocessing to scale data on the features, goal is to get features in between -1 and 1
# svm == support vector machines
# model_selection to create training and testing samples
from sklearn import preprocessing, svm, model_selection
from sklearn.linear_model import LinearRegression

In [2]:
df = quandl.get('WIKI/GOOGL')

In [3]:
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [4]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0


In [5]:
# Create a new column: Percent Volotility
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Low'] * 100
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT
Date,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,8.441017
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,8.537313
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,4.062357
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,7.753210
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,3.966115


In [6]:
# Create column for daily percent change
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT,PCT_change
Date,,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,8.441017,0.324968
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,8.537313,7.227007
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,4.062357,-1.227880
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,7.753210,-5.726357
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,3.966115,1.183658


In [7]:
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0
2004-08-20,54.322689,8.537313,7.227007,22834300.0
2004-08-23,54.869377,4.062357,-1.227880,18256100.0
2004-08-24,52.597363,7.753210,-5.726357,15247300.0
2004-08-25,53.164113,3.966115,1.183658,9188600.0


# Regression Features and Labels

In [10]:
forcast_col = 'Adj. Close'
# in Pandas na can == NaN
# replace NaN with a - in order not to sacrafice data, and treat it as an outlier
df.fillna(-99999, inplace=True)

In [33]:
# Generally we use regression to forcast out
# Here we're trying to predict 1% of the df
forcast_out = int(math.ceil(0.01*len(df)))
print(forcast_out)
# what print is days in advance

34


In [26]:
# columns will be shifted up
# this way the 'labels' column for each row will be the Adj. Close price 1 day into the future
df['labels'] = df[forcast_col].shift(-forcast_out)
df.dropna(inplace=True)
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label,labels
Date,,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849,71.219849


# Regression Training and Testing

In [28]:
# Defined features will be a capital X
# Features are everything except the label column
# df.drop() automatically returns a new df
#  1 == columns, 0 == index
X = np.array(df.drop(['labels'], 1))
# Labels will be a lower case y
y = np.array(df['labels'])

# Here we're scaling X before we feed it through the classifier
X = preprocessing.scale(X)
y = np.array(df['labels'])

print(len(X), len(y))

3319 3319


/var/folders/j3/n_0jvg8n2c7_99cpfggd3kc00000gq/T/ipykernel_59921/2663856110.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(df.drop(['labels'], 1))


In [32]:
# training and testing X and y
# test_size is == to 20% of the data that we want to use as testing data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

# for X_train and y_train, we use to fit our classifiers
classifier = LinearRegression(n_jobs=-1)
# to fit or train a classifier
classifier.fit(X_train, y_train)
# fit is synonomys with train, score is synonomys with test
accuracy = classifier.score(X_test, y_test)

# we want to train and test on seperate data 
# with LinearRegression the accuracy is the squared error
print(accuracy)

1.0


In [37]:
# Using SVM instead of LinearRegression
classifierSVM = svm.SVR(kernel='poly')
accuracySVM = classifier.score(X_test, y_test)
print(accuracySVM)

1.0
